In [2]:
# %load jupyter_default.py
import pandas as pd
import numpy as np
import os
import re
import datetime
import time
import glob
import json
from tqdm import tqdm_notebook
from colorama import Fore, Style

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
from slugify import slugify

%config InlineBackend.figure_format='retina'
sns.set() # Revert to matplotlib defaults
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelpad'] = 20
plt.rcParams['legend.fancybox'] = True
plt.style.use('ggplot')

SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 14, 16, 20
plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=MEDIUM_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE)

def savefig(name):
    plt.savefig(f'../../figures/{name}.png', bbox_inches='tight', dpi=300)

%reload_ext autoreload
%autoreload 2
    
%reload_ext version_information
%version_information pandas, numpy

Software versions
Python 3.6.8 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.4.0
OS Darwin 16.7.0 x86_64 i386 64bit
pandas 0.23.4
numpy 1.14.2
Mon May 06 11:28:04 2019 PDT

In [386]:
from google.cloud import bigquery

In [3]:
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [4]:
client = bigquery.Client()

# Long Term Campaign Value
Alex's development notebook for long term campaign value.

Look for some samples...

In [3]:
def search():
    data = []
    i = 0
    for f_name in glob.glob('../../data/raw/*.jsonl'):
        print('scanning {}'.format(f_name))
        with open(f_name, 'r') as f:
            for line in tqdm_notebook(f.readlines()):
                i += 1
                d = json.loads(line.strip())
                if d['trafficSource']['campaign'] != '(not set)':
                    data.append(d)
                    return data

print(json.dumps(search()))

# with open('/tmp/temp.json', 'w') as f:
#     json.dump(data, f)

scanning ../../data/raw/ga_sessions_20160801.jsonl



[{"visitorId": null, "visitNumber": 2, "visitId": 1470104649, "visitStartTime": 1470104649, "date": "20160801", "totals": {"visits": 1, "hits": 2, "pageviews": 2, "timeOnSite": 7, "bounces": null, "transactions": null, "transactionRevenue": null, "newVisits": null, "screenviews": null, "uniqueScreenviews": null, "timeOnScreen": null, "totalTransactionRevenue": null, "sessionQualityDim": null}, "trafficSource": {"referralPath": null, "campaign": "AW - Electronics", "source": "google", "medium": "cpc", "keyword": "Google Merchandise", "adContent": "Google Merchandise", "adwordsClickInfo": {"campaignId": null, "adGroupId": null, "creativeId": null, "criteriaId": null, "page": 1, "slot": "Top", "criteriaParameters": "not available in demo dataset", "gclId": "Cj0KEQjwh428BRCnvcyI-5nqjY4BEiQAijebwn7y5-AwvT1h3NMJr7ZlQZfe5eDeEpK6qh1xuGd0Z1UaAkbY8P8HAQ", "customerId": null, "adNetworkType": "Google Search", "targetingCriteria": null, "isVideoAd": false}, "isTrueDirect": true, "campaignCode": n

In [7]:
def search():
    data = []
    i = 0
    for f_name in glob.glob('../../data/raw/*.jsonl'):
        print('scanning {}'.format(f_name))
        with open(f_name, 'r') as f:
            for line in tqdm_notebook(f.readlines()):
                i += 1
                d = json.loads(line.strip())
                if d['userId'] is not None:
                    print(d['userId'])
                    print(type(d['userId']))
                    print('Found one!')
                    data.append(d)
                    return data
#                 if d['trafficSource']['campaign'] != '(not set)':
#                     data.append(d)
#                     return data

print(json.dumps(search()))

# with open('/tmp/temp.json', 'w') as f:
#     json.dump(data, f)

scanning ../../data/raw/ga_sessions_20160801.jsonl



scanning ../../data/raw/ga_sessions_20160802.jsonl



scanning ../../data/raw/ga_sessions_20160803.jsonl



scanning ../../data/raw/ga_sessions_20160804.jsonl



scanning ../../data/raw/ga_sessions_20160805.jsonl



scanning ../../data/raw/ga_sessions_20160806.jsonl



scanning ../../data/raw/ga_sessions_20160807.jsonl



scanning ../../data/raw/ga_sessions_20160808.jsonl


KeyboardInterrupt: 

OK so there's no user ID in this dataset.

What is `fullVisitorId`

In [9]:
def search():
    data = []
    i = 0
    for f_name in glob.glob('../../data/raw/*.jsonl'):
        print('scanning {}'.format(f_name))
        with open(f_name, 'r') as f:
            for line in tqdm_notebook(f.readlines()):
                i += 1
                print(i)
                d = json.loads(line.strip())
                if d['fullVisitorId'] in data:
                    print('Repeat {}'.format(d['fullVisitorId']))
                    return {}
                data.append(d['fullVisitorId'])
                

print(json.dumps(search()))

# with open('/tmp/temp.json', 'w') as f:
#     json.dump(data, f)

scanning ../../data/raw/ga_sessions_20160801.jsonl


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Repeat 8159312408158297118

{}


Let's use this as a unique ID

## Read data

### Read from bigquery (old code)

In [49]:
%%time
"""
Using bigquery
"""

def pull_daily_data(verbose=False):
    dataset = client.get_dataset('bigquery-public-data.google_analytics_sample')

    data = []
    for table in tqdm_notebook(list(client.list_tables(dataset))):
        if verbose:
            print('Querying {}'.format(table.table_id))
        query_job = client.query('''
            select
              date,
              sum(totals.visits),
              sum(totals.pageviews),
              sum(totals.transactions),
              sum(totals.transactionRevenue)
            from `bigquery-public-data.google_analytics_sample.{}`
            group by date;
        '''.format(table.table_id))
        results = query_job.result().to_dataframe()
        results.columns = ['date', 'visits', 'pageviews', 'transactions', 'transactionRevenue']
        data.append(results)

    df = pd.concat(data, ignore_index=True, sort=False)
    return df

bq_results = pull_daily_data()


CPU times: user 7.94 s, sys: 856 ms, total: 8.8 s
Wall time: 11min 29s


In [53]:
bq_results.head()

,date,visits,pageviews,transactions,transactionRevenue
0,20160801,1711,9843,34,6116060000
1,20160802,2140,11784,18,1361190000
2,20160803,2890,13724,None,None
3,20160804,3161,13326,17,1182890000
4,20160805,2702,13585,42,5594260000


### Read `jsonl` from google drive (old code)

In [7]:
%%time
"""
Using gdrive jsonl
"""
ERRORS = []

def pull_daily_data(verbose=False, raise_errors=False):
    dataset = sorted(glob.glob('/Volumes/GoogleDrive/My Drive/bigquery_ga_sample/*.jsonl'))

    data = []
    for table in tqdm_notebook(dataset):
        if verbose:
            print('Scanning {}'.format(table))
        with open(table, 'r') as f:
            table_data = []
            for line in f:
                d = json.loads(line)
                date = d['date']
                d = d['totals']
                try:
                    table_data.append([
                        date,
                        d['visits'],
                        d['pageviews'],
                        d['transactions'],
                        d['transactionRevenue'],
                    ])
                except Exception as e:
                    if verbose:
                        print('Error raised when reading row:\n{}'.format(e))
                    ERRORS.append([table, e])
                    if raise_errors:
                        raise(e)

            cols = ['date', 'visits', 'pageviews', 'transactions', 'transactionRevenue']
            results = (
                pd.DataFrame(table_data, columns=cols)
                    .groupby('date')[['visits', 'pageviews', 'transactions', 'transactionRevenue']]
                    .sum().reset_index()
            )
            data.append(results)

    df = pd.concat(data, ignore_index=True, sort=False)
    return df

jsonl_gdrive_results = pull_daily_data()


CPU times: user 7min 55s, sys: 1min 56s, total: 9min 52s
Wall time: 23min 14s


In [8]:
jsonl_gdrive_results

,date,visits,pageviews,transactions,transactionRevenue
0,20160801,1711,9843.0,34.0,6.116060e+09
1,20160802,2140,11784.0,18.0,1.361190e+09
2,20160803,2890,13724.0,NaN,NaN
3,20160804,3161,13326.0,17.0,1.182890e+09
4,20160805,2702,13585.0,42.0,5.594260e+09
5,20160806,1663,6926.0,10.0,1.891040e+09
6,20160807,1622,7239.0,16.0,2.410730e+09
7,20160808,2815,13179.0,36.0,5.273810e+09
8,20160809,2851,13411.0,44.0,5.596400e+09
9,20160810,2757,13003.0,46.0,4.284210e+09


### Read `jsonl` from local

In [36]:
sorted(glob.glob('../../data/raw/*.jsonl'))

['../../data/raw/ga_sessions_20160801.jsonl',
 '../../data/raw/ga_sessions_20160802.jsonl',
 '../../data/raw/ga_sessions_20160803.jsonl',
 '../../data/raw/ga_sessions_20160804.jsonl',
 '../../data/raw/ga_sessions_20160805.jsonl',
 '../../data/raw/ga_sessions_20160806.jsonl',
 '../../data/raw/ga_sessions_20160807.jsonl',
 '../../data/raw/ga_sessions_20160808.jsonl',
 '../../data/raw/ga_sessions_20160809.jsonl',
 '../../data/raw/ga_sessions_20160810.jsonl',
 '../../data/raw/ga_sessions_20160811.jsonl',
 '../../data/raw/ga_sessions_20160812.jsonl',
 '../../data/raw/ga_sessions_20160813.jsonl',
 '../../data/raw/ga_sessions_20160814.jsonl',
 '../../data/raw/ga_sessions_20160815.jsonl',
 '../../data/raw/ga_sessions_20160816.jsonl',
 '../../data/raw/ga_sessions_20160817.jsonl',
 '../../data/raw/ga_sessions_20160818.jsonl',
 '../../data/raw/ga_sessions_20160819.jsonl',
 '../../data/raw/ga_sessions_20160820.jsonl',
 '../../data/raw/ga_sessions_20160821.jsonl',
 '../../data/raw/ga_sessions_20160

In [20]:
%%time
"""
Using local jsonl
"""
ERRORS = []

def pull_daily_data(verbose=False, raise_errors=False):
    dataset = sorted(glob.glob('../../data/raw/*.jsonl'))

    data = []
    for table in tqdm_notebook(dataset):
        if verbose:
            print('Scanning {}'.format(table))
        with open(table, 'r') as f:
            table_data = []
            visitor_id_dates = {}
            for line in f:
                try:
                    d = json.loads(line)
                    
                    if d['trafficSource']['campaign'] != '(not set)':
                        new_acquisition = True
                        date_acquired = float('nan')
                        table_data.append([
                            d['date'],
                            d['userId'],
                            d['fullVisitorId'],
                            d['trafficSource']['campaign'],
                            d['trafficSource']['source'],
                            d['trafficSource']['medium'],
                            d['totals']['transactionRevenue'],
                            new_acquisition,
                            date_acquired,
                        ])
                        visitor_id_dates[d['fullVisitorId']] = d['date']
                        
                    elif d['fullVisitorId'] in visitor_id_dates.keys():
                        new_acquisition = False
                        date_acquired = visitor_id_dates[d['fullVisitorId']]
                        table_data.append([
                            d['date'],
                            d['userId'],
                            d['fullVisitorId'],
                            d['trafficSource']['campaign'],
                            d['trafficSource']['source'],
                            d['trafficSource']['medium'],
                            d['totals']['transactionRevenue'],
                            new_acquisition,
                            date_acquired,
                        ])
                        
                except Exception as e:
                    if verbose:
                        print('Error raised when reading row:\n{}'.format(e))
                    ERRORS.append([table, e])
                    if raise_errors:
                        raise(e)

            cols = ['date', 'userId', 'fullVisitorId', 'campaign', 'source', 'medium', 'transactionRevenue', 'new_acquisition', 'date_acquired']
            results = pd.DataFrame(table_data, columns=cols)
            data.append(results)

    df = pd.concat(data, ignore_index=True, sort=False)
    return df

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 13.1 µs


In [21]:
jsonl_results = pull_daily_data(raise_errors=True)

KeyboardInterrupt: 

In [22]:
jsonl_results = pull_daily_data()

In [23]:
df = jsonl_results.copy()
df.date = pd.to_datetime(df.date)
df.to_csv('../../data/interim/long_term_campaign_value_raw.csv', index=False)

In [39]:
def load_file(f_path):
    if not os.path.exists(f_path):
        print('No data found. Run data load script above.')
        return
    print('Loading {}'.format(f_path))
    df = pd.read_csv(f_path)
    df.date = pd.to_datetime(df.date)
    return df

# Looking forward to walrus operator for stuff like this...
tmp = load_file('../../data/interim/long_term_campaign_value_raw.csv')
if tmp is not None:
    print('Loading from file')
    df = tmp.copy()
    del tmp

Loading ../../data/interim/long_term_campaign_value_raw.csv
Loading from file


In [40]:
df.head()

,date,userId,fullVisitorId,campaign,source,medium,transactionRevenue,new_acquisition,date_acquired
0,2016-08-01,NaN,721240306323694297,AW - Electronics,google,cpc,NaN,True,NaN
1,2016-08-01,NaN,793874482369095202,Data Share Promo,Partners,affiliate,NaN,True,NaN
2,2016-08-01,NaN,5716663734226378541,Data Share Promo,Partners,affiliate,NaN,True,NaN
3,2016-08-01,NaN,6091498163535424271,AW - Electronics,google,cpc,NaN,True,NaN
4,2016-08-01,NaN,1407445369428897300,AW - Electronics,google,cpc,NaN,True,NaN


In [41]:
# df['week'] = df.date.apply(lambda x: x.strftime('%W'))
# df['year'] = df.date.apply(lambda x: x.strftime('%Y'))
# df['week_start'] = df[['week', 'year']].apply(
#     lambda x: datetime.datetime.strptime('{}-{}-1'.format(x.year, x.week), '%Y-%W-%w'),
#     axis=1
# )

In [42]:
df.dtypes

date                  datetime64[ns]
userId                       float64
fullVisitorId                 uint64
campaign                      object
source                        object
medium                        object
transactionRevenue           float64
new_acquisition                 bool
date_acquired                float64
dtype: object

## Exploring the dataset I was able to pull

In [43]:
df['date'] = pd.to_datetime(df['date'])
(df.new_acquisition == False).sum()

1608

1.6k sessions from acqusition efforts

In [45]:
m = df.new_acquisition == False
df_long_term = df[m].copy()

df_long_term['date_delta'] = df_long_term.date_acquired.apply(
    lambda x: datetime.datetime.strptime('{:.0f}'.format(x), '%Y%m%d')
) - df_long_term['date']

In [46]:
df_long_term

,date,userId,fullVisitorId,campaign,source,medium,transactionRevenue,new_acquisition,date_acquired,date_delta
135,2016-08-03,NaN,7656785592173575873,(not set),google,organic,NaN,False,20160803.0,0 days
193,2016-08-03,NaN,179435387805366273,(not set),analytics.google.com,referral,NaN,False,20160803.0,0 days
256,2016-08-04,NaN,1857000194971924809,(not set),google,organic,NaN,False,20160804.0,0 days
307,2016-08-04,NaN,7478564696098649370,(not set),google,organic,NaN,False,20160804.0,0 days
351,2016-08-04,NaN,4812040237159174207,(not set),google,organic,NaN,False,20160804.0,0 days
362,2016-08-04,NaN,6124193044741214901,(not set),analytics.google.com,referral,NaN,False,20160804.0,0 days
428,2016-08-04,NaN,3765840250345504147,(not set),(direct),(none),NaN,False,20160804.0,0 days
488,2016-08-04,NaN,6180363097948659148,(not set),analytics.google.com,referral,NaN,False,20160804.0,0 days
497,2016-08-04,NaN,6124193044741214901,(not set),analytics.google.com,referral,NaN,False,20160804.0,0 days
500,2016-08-04,NaN,8635849898236107532,(not set),google,organic,NaN,False,20160804.0,0 days


In [47]:
df_long_term.date_delta.value_counts()

0 days    1608
Name: date_delta, dtype: int64

Well... something is not working here. I don't think `fullVisitorId` will work for us.

In [26]:
from IPython.display import HTML
HTML('<style>div.text_cell_render{font-size:130%;padding-top:50px;padding-bottom:50px}</style>')